In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image

In [3]:
!pip install --upgrade scikit-learn

     |████████████████████████████████| 24.9 MB 18.2 MB/s eta 0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.2
    Uninstalling scikit-learn-0.23.2:
      Successfully uninstalled scikit-learn-0.23.2


In [4]:
article_tags = pd.read_csv("articles_tags.csv")
article_text = pd.read_csv("../resources/articles_text.csv")
article_tags= article_tags.set_index("Resource URL")
article_text = article_text.set_index("Resource URL")
article_text = article_text[~article_text.index.duplicated(keep='first')]

In [5]:
for i in article_text.index:
    article_text.loc[i, "tech_tag"] =  article_tags.loc[i]["Technical (Y/N)"][0]
    if article_text.loc[i, "tech_tag"] not in ["Y", "N"]:
        article_text.loc[i, "tech_tag"]  = "M"

In [6]:
data = (article_text[(article_text["tech_tag"]=="Y") | (article_text["tech_tag"]=="N")])
data_src = data
data_src = data_src[["Resource Title", "text", "tech_tag"]]
data_src.loc[:, "text"] =  data_src["Resource Title"] + " " +data_src["text"]
data_src.loc[:, "tech_tag"] = data_src["tech_tag"].map({"N": 0, "Y":1})
data_src.drop("Resource Title", axis=1, inplace=True)
data_src = data_src.dropna().reset_index(drop=True)
data=data_src

/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1783: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:4162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [7]:
unlabelled_data = (article_text[(article_text["tech_tag"]!="Y") & (article_text["tech_tag"]!="N")])
data_src = unlabelled_data
data_src = data_src[["Resource Title", "text"]]
data_src.loc[:, "text"] =  data_src["Resource Title"] + " " +data_src["text"]
data_src.loc[:, "tech_tag"] = -1
data_src.drop("Resource Title", axis=1, inplace=True)
data_src = data_src.dropna().reset_index(drop=True)
unlabelled_data=data_src

/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1783: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [25]:
import os

import numpy as np

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

# Parameters
sdg_params = dict(alpha=1e-5, penalty='l2', loss='log')
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)

# Supervised Pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(**vectorizer_params)),
    ('tfidf', TfidfTransformer()),
    ('clf', MLPClassifier(alpha=1, max_iter=1000), verbose=True),
])
# SelfTraining Pipeline
st_pipeline = Pipeline([
    ('vect', CountVectorizer(**vectorizer_params)),
    ('tfidf', TfidfTransformer()),
    ('clf', SelfTrainingClassifier(MLPClassifier(alpha=1, max_iter=1000), verbose=True)),
])
# LabelSpreading Pipeline
ls_pipeline = Pipeline([
    ('vect', CountVectorizer(**vectorizer_params)),
    ('tfidf', TfidfTransformer()),
    # LabelSpreading does not support dense matrices
    ('todense', FunctionTransformer(lambda x: x.todense())),
    ('clf', LabelSpreading()),
])


def eval_and_print_metrics(clf, X_train, y_train, X_test, y_test):
    print("Number of training samples:", len(X_train))
    print("Unlabeled samples in training set:",
          sum(1 for x in y_train if x == -1))
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print ('Accuracy Score :',accuracy_score(y_test, y_pred))
    print("Micro-averaged F1 score on test set: "
          "%0.3f" % f1_score(y_test, y_pred, average='micro'))
    print("-" * 10)
    print()




SyntaxError: invalid syntax (<ipython-input-25-467913dd0a2f>, line 26)

In [18]:
X, y = data.text, data.tech_tag
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size = 0.5)

print("Supervised SGDClassifier on 100% of the data:")
eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

# select a mask of 20% of the train dataset
y_mask = np.random.rand(len(y_train)) < 0.2

# X_20 and y_20 are the subset of the train dataset indicated by the mask
X_20, y_20 = map(list, zip(*((x, y)
                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
print("Supervised SGDClassifier on 20% of the training data:")
eval_and_print_metrics(pipeline, X_20, y_20, X_test, y_test)

# set the non-masked subset to be unlabeled
y_train[~y_mask] = -1
print("SelfTrainingClassifier on 20% of the training data (rest "
      "is unlabeled):")
eval_and_print_metrics(st_pipeline, X_train, y_train, X_test, y_test)

if 'CI' not in os.environ:
    # LabelSpreading takes too long to run in the online documentation
    print("LabelSpreading on 20% of the data (rest is unlabeled):")
    eval_and_print_metrics(ls_pipeline, X_train, y_train, X_test, y_test)

Supervised SGDClassifier on 100% of the data:
Number of training samples: 500
Unlabeled samples in training set: 0
Accuracy Score : 0.918
Micro-averaged F1 score on test set: 0.918
----------

Supervised SGDClassifier on 20% of the training data:
Number of training samples: 97
Unlabeled samples in training set: 0
Accuracy Score : 0.844
Micro-averaged F1 score on test set: 0.844
----------

SelfTrainingClassifier on 20% of the training data (rest is unlabeled):
Number of training samples: 500
Unlabeled samples in training set: 403
End of iteration 1, added 225 new labels.
End of iteration 2, added 60 new labels.
End of iteration 3, added 17 new labels.
End of iteration 4, added 4 new labels.
Accuracy Score : 0.764
Micro-averaged F1 score on test set: 0.764
----------

LabelSpreading on 20% of the data (rest is unlabeled):
Number of training samples: 500
Unlabeled samples in training set: 403
Accuracy Score : 0.872
Micro-averaged F1 score on test set: 0.872
----------



In [19]:
X, y = data.text, data.tech_tag
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size = 0.5)

In [20]:
len(X_train)

500

In [21]:
X_train = pd.concat([X_train, unlabelled_data.text], ignore_index = True)
y_train = pd.concat([y_train, unlabelled_data.tech_tag], ignore_index = True)

In [22]:
len(y_train)

1695

In [23]:
print("LabelSpreading on 20% of the data (rest is unlabeled):")
eval_and_print_metrics(ls_pipeline, X_train, y_train, X_test, y_test)

LabelSpreading on 20% of the data (rest is unlabeled):
Number of training samples: 1695
Unlabeled samples in training set: 1195
Accuracy Score : 0.722
Micro-averaged F1 score on test set: 0.722
----------



In [24]:
print("SelfTrainingClassifier on 20% of the training data (rest "
      "is unlabeled):")
eval_and_print_metrics(st_pipeline, X_train, y_train, X_test, y_test)

SelfTrainingClassifier on 20% of the training data (rest is unlabeled):
Number of training samples: 1695
Unlabeled samples in training set: 1195
End of iteration 1, added 735 new labels.
End of iteration 2, added 257 new labels.
End of iteration 3, added 92 new labels.
End of iteration 4, added 26 new labels.
End of iteration 5, added 5 new labels.
Accuracy Score : 0.838
Micro-averaged F1 score on test set: 0.838
----------

